[![Ask me on Discord](https://img.shields.io/badge/Ask%20me%20on%20Discord-blue?style=for-the-badge&logo=https://simpleicons.org/icons/discord.svg&link=https://lookup.guru/512044995696525322)](https://lookup.guru/512044995696525322)  [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=for-the-badge)](https://colab.research.google.com/github/DEX-1101/sd-webui-notebook/blob/main/kaggle-colab-notebook.ipynb) [![Visitors](https://api.visitorbadge.io/api/visitors?path=https%3A%2F%2Fgithub.com%2FDEX-1101%2Fsd-webui-notebook%2Fblob%2Fmain%2Fkaggle-colab-notebook.ipynb&countColor=%2337d67a)](https://visitorbadge.io/status?path=https%3A%2F%2Fgithub.com%2FDEX-1101%2Fsd-webui-notebook%2Fblob%2Fmain%2Fkaggle-colab-notebook.ipynb)

In [ ]:
import_config = "https://raw.githubusercontent.com/DEX-1101/sd-webui-notebook/main/res/config.json" # import your webui config here, if have one

#=====================================================
!pip install -q git+https://github.com/DEX-1101/colablib &> /dev/null 
!curl -s -OL https://github.com/DEX-1101/sd-webui-notebook/raw/dev/res/x1101.py
!curl -s -OL https://github.com/DEX-1101/sd-webui-notebook/raw/dev/res/pastebin_dl.py
!curl -s -OL https://github.com/DEX-1101/sd-webui-notebook/raw/dev/res/get_ip.py
    
%run x1101.py
!wget -q {import_config} -O {ui}/config.json

# Custom code here ================
!git clone -q https://github.com/Zyin055/Config-Presets {ui}/x1101/extensions/config-presets
!wget -q https://github.com/DEX-1101/sd-webui-notebook/raw/main/res/config-txt2img.json -O {ui}/x1101/extensions/config-presets/config-txt2img.json
!wget -q https://huggingface.co/x1101/UI/resolve/main/4x-UltraSharp.pth -O {ui}/x1101/models/ESRGAN/4x-UltraSharp.pth
# =================================

### Download from Pastebin URL

In [ ]:
pastebin_url = "https://pastebin.com/mr0v09pG" # ecample link
hf_token = "" # if use private repo

#=====================================================
pasta = f"sed -i 's|pastebin_url = \"\"|pastebin_url = \"{pastebin_url}\"|g; s|hf_token = \"\"|hf_token = \"{hf_token}\"|g' {ui}/pastebin_dl.py"
os.system(pasta)
%run {ui}/pastebin_dl.py

## Manual Download

In [ ]:
civitai_api = "" # Put your api key here if error downloading from civitai

#================================================
oppai = f"sed -i 's|toket = \"\"|toket = \"?token={civitai_api}\"|g' {ui}/semvak_zeus.py"
os.system(oppai)
%run {ui}/semvak_zeus.py

# Models
%cd {ui}/x1101/models/Stable-diffusion
%download https://civitai.com/api/download/models/275408

# Lora
%cd {ui}/x1101/models/Lora
%download https://huggingface.co/x1101/sdxl-model/resolve/main/acheron/acheron/acheron.safetensors

# Embeddings   
%cd {ui}/x1101/embeddings
%download https://civitai.com/api/download/models/245812

# ControlNet    
%cd {ui}/x1101/extensions/sd-webui-controlnet/models/
%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml

## Run WebUI

In [ ]:
ngrok_token = "" # https://dashboard.ngrok.com/get-started/your-authtoken
zrok_token = "" # optional, good ngrok alternative
password = "x1101"

#================================================
%cd {ui}
%run get_ip.py
tunnel.add_tunnel(command="cl tunnel --url localhost:{port}", name="cl", pattern=re.compile(r"[\w-]+\.trycloudflare\.com"))
tunnel.add_tunnel(command="lt --port {port}", name="lt", pattern=re.compile(r"[\w-]+\.loca\.lt"), note="Password : " + Fore.GREEN + public_ipv4 + Style.RESET_ALL + " rerun cell if 404 error.")
if zrok_token:
    !zrok enable {zrok_token} &> /dev/null
    tunnel.add_tunnel(command="zrok share public http://localhost:{port}/ --headless", name="zrok", pattern=re.compile(r"[\w-]+\.share\.zrok\.io"))

with tunnel:
    #!python -m http.server 1101
    %cd {ui}/x1101
    !echo -n {start_colab} >{ui}/x1101/static/colabTimer.txt
    !sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' {ui}/x1101/modules/shared_options.py
    !python launch.py --port=1101 --ngrok {ngrok_token}  --encrypt-pass={password} --xformers --theme dark --enable-insecure-extension-access --disable-console-progressbars --disable-safe-unpickle --no-half-vae